In [1]:
import pandas as pd

# Load the dataset
data_path = 'CollinsEtAl_2018_PNAS.csv'
data = pd.read_csv(data_path)

# Display the first few rows of the dataset
print(data.head())

# Display basic statistics of the dataset
print(data.describe())

# Display the column names
print(data.columns)

   subno  block  ns  time  stimseq  imageseq  folderseq  iterseq  corAseq  \
0      1      1   5     1        1         1          1        1        1   
1      1      1   5     2        5         5          1        1        3   
2      1      1   5     3        3         3          1        1        2   
3      1      1   5     4        1         1          1        2        1   
4      1      1   5     5        2         2          1        1        1   

   choice  key  cor  rew        rt  condition (HC=0,SZ=1)  pcor  delay  
0       2   13    0    0  0.758829                      1   NaN    NaN  
1       1   14    0    0  0.678557                      1   NaN    NaN  
2       1   14    0    0  0.584314                      1   NaN    NaN  
3      -1   -1   -1   -1 -1.000000                      1   0.0    NaN  
4       3   15    0    0  0.782260                      1   NaN    NaN  
              subno         block            ns          time       stimseq  \
count  30403.000000 

In [2]:
# Extract relevant columns
columns = ['subno', 'block', 'ns', 'stimseq', 'corAseq', 'choice', 'cor', 'rew', 'rt', 'pcor', 'delay']
df = data[columns]

# Display the first few rows of the extracted data
print(df.head())


   subno  block  ns  stimseq  corAseq  choice  cor  rew        rt  pcor  delay
0      1      1   5        1        1       2    0    0  0.758829   NaN    NaN
1      1      1   5        5        3       1    0    0  0.678557   NaN    NaN
2      1      1   5        3        2       1    0    0  0.584314   NaN    NaN
3      1      1   5        1        1      -1   -1   -1 -1.000000   0.0    NaN
4      1      1   5        2        1       3    0    0  0.782260   NaN    NaN


In [3]:
import numpy as np
from scipy.optimize import minimize

def softmax(Q, s, beta):
    exp_Q = np.exp(beta * Q[s])
    return exp_Q / np.sum(exp_Q)

def q_learning_update(Q, s, a, r, alpha):
    prediction_error = r - Q[s, a]
    Q[s, a] += alpha * prediction_error
    return Q

def wm_update(W, s, a, r, phi):
    W[s, a] = r
    return W

def rlwm_model(params, actions, stimuli, rewards, num_states, num_actions):
    alpha, beta, epsilon, bias, K, phi, rho = params
    Q = np.zeros((num_states, num_actions))
    W = np.zeros((num_states, num_actions))
    log_likelihood = 0

    for t in range(len(actions)):
        s, a, r = stimuli[t], actions[t], rewards[t]
        wm_contrib = min(1, K / num_states)
        mixed_policy = rho * softmax(W, s, beta) + (1 - rho) * softmax(Q, s, beta)
        action_prob = (1 - epsilon) * mixed_policy[a] + epsilon / num_actions
        log_likelihood += np.log(action_prob)

        if r > 0:
            alpha_adjusted = alpha * (1 - bias)
        else:
            alpha_adjusted = alpha

        Q = q_learning_update(Q, s, a, r, alpha_adjusted)
        W = wm_update(W, s, a, r, phi)

    return -log_likelihood  # Negative log likelihood for optimization


In [4]:
# Prepare data
subjects = df['subno'].unique()
num_states = df['stimseq'].nunique()
num_actions = df['corAseq'].nunique()

# Define the objective function for optimization
def objective_function(params, df):
    log_likelihood = 0
    for subno in subjects:
        sub_df = df[df['subno'] == subno]
        actions = sub_df['choice'].values
        stimuli = sub_df['stimseq'].values
        rewards = sub_df['rew'].values
        log_likelihood += rlwm_model(params, actions, stimuli, rewards, num_states, num_actions)
    return log_likelihood

# Initial guess for parameters
initial_params = [0.1, 5.0, 0.05, 0.1, 3, 0.1, 0.5]

# Optimize parameters
result = minimize(objective_function, initial_params, args=(df,))
print("Fitted parameters:", result.x)
print("Log likelihood:", -result.fun)


IndexError: index 3 is out of bounds for axis 0 with size 3

In [5]:
print("Unique actions:", df['choice'].unique())
print("Unique stimuli:", df['stimseq'].unique())
print("Unique rewards:", df['rew'].unique())


Unique actions: [ 2  1 -1  3]
Unique stimuli: [1 5 3 2 4 6]
Unique rewards: [ 0 -1  1]


In [10]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Load the dataset
data_path = 'CollinsEtAl_2018_PNAS.csv'
df = pd.read_csv(data_path)

# Extract relevant columns
columns = ['subno', 'block', 'ns', 'stimseq', 'corAseq', 'choice', 'cor', 'rew', 'rt', 'pcor', 'delay']
df = df[columns]

# Ensure action values are within the valid range
df = df[df['choice'] >= 0]  # Remove invalid actions
df = df[df['choice'] < df['corAseq'].nunique()]  # Ensure actions are within valid range

# Parameters
subjects = df['subno'].unique()
num_states = df['stimseq'].nunique()
num_actions = df['corAseq'].nunique()

def softmax(Q, s, beta):
    exp_Q = np.exp(beta * Q[s])
    return exp_Q / np.sum(exp_Q)

def q_learning_update(Q, s, a, r, alpha):
    prediction_error = r - Q[s, a]
    Q[s, a] += alpha * prediction_error
    return Q

def wm_update(W, s, a, r, phi):
    W[s, a] = r
    return W

def rlwm_model(params, actions, stimuli, rewards, num_states, num_actions):
    alpha, beta, epsilon, bias, K, phi, rho = params
    Q = np.zeros((num_states, num_actions))
    W = np.zeros((num_states, num_actions))
    log_likelihood = 0

    for t in range(len(actions)):
        s, a, r = stimuli[t], actions[t], rewards[t]

        # Ensure state and action indices are within valid range
        if s >= num_states or s < 0 or a >= num_actions or a < 0:
            continue  # Skip invalid states or actions

        wm_contrib = min(1, K / num_states)
        mixed_policy = rho * softmax(W, s, beta) + (1 - rho) * softmax(Q, s, beta)
        action_prob = (1 - epsilon) * mixed_policy[a] + epsilon / num_actions

        # Ensure action probability is valid
        if action_prob <= 0 or action_prob > 1:
            continue  # Skip invalid action probabilities

        log_likelihood += np.log(action_prob)

        if r > 0:
            alpha_adjusted = alpha * (1 - bias)
        else:
            alpha_adjusted = alpha

        Q = q_learning_update(Q, s, a, r, alpha_adjusted)
        W = wm_update(W, s, a, r, phi)

    return -log_likelihood  # Negative log likelihood for optimization

# Define the objective function for optimization
def objective_function(params, df):
    log_likelihood = 0
    for subno in subjects:
        sub_df = df[df['subno'] == subno]
        actions = sub_df['choice'].values
        stimuli = sub_df['stimseq'].values
        rewards = sub_df['rew'].values
        log_likelihood += rlwm_model(params, actions, stimuli, rewards, num_states, num_actions)
    return log_likelihood

# Initial guess for parameters
initial_params = [0.1, 5.0, 0.05, 0.1, 3, 0.1, 0.5]

# Optimize parameters
result = minimize(objective_function, initial_params, args=(df,))
print("Fitted parameters:", result.x)
print("Log likelihood:", -result.fun)


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/CollinsEtAl_2018_PNAS.csv'